#### Eyelids Blink  Patten Password Authentication System

In [1]:
import import_ipynb
import time
import cv2 
import numpy as np
import pandas as pd
from imutils.video import WebcamVideoStream

open_eye_Classifier = cv2.CascadeClassifier("Classifiers/haarcascade_eye.xml")
left_eye_Classifier = cv2.CascadeClassifier("Classifiers/haarcascade_lefteye_2splits.xml")
right_eye_Classifier = cv2.CascadeClassifier("Classifiers/haarcascade_righteye_2splits.xml")

# Values initialization
l_found = False
r_found = False

l_x = 0
l_y = 0
r_x = 0
r_y = 0
f_x = 0
f_y = 0

# Left and Right Eyes State :  "open","closed"
# EyesStatePattern(L,R,B,O): L:left closed R:right closed B:both closed O:both open
frame_temp = ''

left_eye_state = 'close'
right_eye_state = 'close'

counter=0
eye_states_pattern = ''
eye_states_count = 0

scale = 1.3
frame = ''

def detect_left_and_right_eyes(frame):
    # Detecting the Left Eye -- then the eft most eyes
    global l_found,r_found,l_x,l_y,r_x,r_y,f_x,f_y,left_eye_state,right_eye_state,scale,frame_temp
    l_eye_coor = left_eye_Classifier.detectMultiScale(frame,scaleFactor=scale,minNeighbors=5)
    l_found = False
    leftmost_eyes=()
    biggest_area = 0
    i=0
    for (x,y,w,h) in l_eye_coor:
        l_found = True
        if i == 0:
            leftmost_eyes= (x,y,w,h)
            biggest_area = w*h
        elif x>leftmost_eyes[0]:
            leftmost_eyes=(x,y,w,h)
            biggest_area = w*h
        i=i+1
    # Drawing the rectangle for Left eye
    if l_found==True:
        x,y,w,h = leftmost_eyes
        cv2.rectangle(frame_temp,(x-10,y-10),(x+w+10,y+h+10),(0,0,0),2)
        l_x=x
        l_y=y
    # Detecting the Right Eyes -- then teh right most eyes
    
    r_eye_coor = right_eye_Classifier.detectMultiScale(frame,scaleFactor=scale,minNeighbors=5)
    r_found = False
    rightmost_eyes =()
    biggest_area = 0
    i=0
    
    for (x,y,w,h) in r_eye_coor:
        r_found = True
        if i == 0:
            rightmost_eyes = (x,y,w,h)
            biggest_area = w*h
        elif x<rightmost_eyes[0]:
            rightmost_eyes=(x,y,w,h)
            biggest_area = w*h
        i=i+1
    # Drawing the rectangle for Left eye
    if r_found==True:
        x,y,w,h = rightmost_eyes
        cv2.rectangle(frame_temp,(x-10,y-10),(x+w+10,y+h+10),(0,0,0),2)
        r_x = x
        r_y = y
    return 0

def detect_open_eyes(frame):
    global l_found,r_found,l_x,l_y,r_x,r_y,scale,left_eye_state,right_eye_state,scale,frame_temp
    left_eye_state = 'close'
    right_eye_state = 'close'
    
    eyes_coor = open_eye_Classifier.detectMultiScale(frame,scaleFactor=scale,minNeighbors=40)
   
    i=0
    #If eyes are open --> Draw rectangle not open --> Don't draw rectangle
    for (x,y,w,h) in eyes_coor:
        if(i>=2):
            break
        #Draw the rectangle
        cv2.rectangle(frame_temp,(x,y),(x+w,y+h),(255, 255, 0),2)
        # Check if Left Eye Open
        
        if abs(l_x - x)<f_x*0.055 and abs(l_y - y)<f_y*0.055 and l_found == True:
            left_eye_state = "open"
        # Check if Right Eye Open
        if abs(r_x - x)<f_x*0.055 and abs(r_y - y)<f_y*0.055 and r_found == True:
            right_eye_state = "open"
        i=i+1
    return left_eye_state,right_eye_state,frame

global Tempstate



'''REGISTRATION OF PASSKEY !!!!! '''

start = input("\n\nREGISTRATION: Enter 's' start the processing password and 'q' to stop recording password.\n\n") 

count = 8
k=0

if start=='s':
    cap=cv2.VideoCapture(0)
    password = []
    
    index = 0
    global last_code
    
    last_code = ''
    counter = 0
    
    while True:
        time.sleep(0.1)
        ret,frame = cap.read()
        #frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        
        frame_temp = frame.copy()
        
        print(frame)
        print(type(frame))
        print(frame.shape)
        
        if counter == 0:
            f_x = frame.shape[0]
            f_y = frame.shape[1]
        if cv2.waitKey(1) == ord('q'):
            break
    
        detect_left_and_right_eyes(frame)
        detect_open_eyes(frame)
        
        cv2.imshow('EYE BLINKING AUTHENTICATION',frame_temp)
        
        #Look for the closed eyes
        if l_found == True and r_found == True:
        
            if(left_eye_state == "open" and right_eye_state=="open"):
                Tempstate = 'OO' #
            if(left_eye_state == "open" and right_eye_state=="close"):
                Tempstate = 'RC'
            if(left_eye_state == "close" and right_eye_state=="open"):
                Tempstate = 'LC'
            if(left_eye_state == "close" and right_eye_state=="close"):
                Tempstate = 'BC'
            
            temp_passkey = Tempstate
            print(left_eye_state,right_eye_state," : ",temp_passkey)
            
            
            if index == 0:
                if k<count:
                    k+=1
                    continue
                k=0
                
                password.append(temp_passkey)
                last_code = temp_passkey
                index+=1
             
            elif index!=0 and temp_passkey!=last_code:
                if k<count:
                    k+=1
                    continue
                k=0
                password.append(temp_passkey)
                last_code = temp_passkey
                index+=1
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
      
    cap.release()
    cv2.destroyAllWindows()
    print("\nYour Registered Password Sequence is: \n",password)



REGISTRATION: Enter 's' start the processing password and 'q' to stop recording password.

s
[[[ 67  55  57]
  [ 65  53  55]
  [ 58  53  54]
  ...
  [ 27  25  30]
  [ 20  24  28]
  [ 11  14  18]]

 [[ 60  52  54]
  [ 61  54  55]
  [ 57  49  53]
  ...
  [ 21  20  23]
  [ 30  29  33]
  [ 16  15  19]]

 [[ 61  48  53]
  [ 61  48  53]
  [ 59  46  50]
  ...
  [ 28  27  30]
  [ 30  29  32]
  [ 26  25  27]]

 ...

 [[154 137 147]
  [154 137 147]
  [154 138 145]
  ...
  [ 41  29  24]
  [ 41  29  24]
  [ 42  30  25]]

 [[152 138 145]
  [151 137 144]
  [154 135 143]
  ...
  [ 40  27  25]
  [ 42  27  25]
  [ 42  27  25]]

 [[153 137 144]
  [153 137 144]
  [152 136 143]
  ...
  [ 38  25  22]
  [ 38  25  22]
  [ 38  25  22]]]
<class 'numpy.ndarray'>
(480, 640, 3)
[[[ 91  53  44]
  [ 86  49  39]
  [ 97  56  36]
  ...
  [ 39  28  12]
  [ 47  27  12]
  [ 48  28  13]]

 [[ 93  57  43]
  [ 93  57  43]
  [107  68  47]
  ...
  [ 41  25  16]
  [ 41  28  12]
  [ 41  28  12]]

 [[101  60  39]
  [105  64  4

close close  :  BC
[[[ 48  53  52]
  [ 45  50  49]
  [ 48  46  51]
  ...
  [ 25  15  23]
  [ 19  15  22]
  [ 22  17  25]]

 [[ 51  51  51]
  [ 51  51  51]
  [ 49  47  52]
  ...
  [ 24  17  25]
  [ 22  17  25]
  [ 21  16  23]]

 [[ 46  44  44]
  [ 50  47  48]
  [ 54  52  57]
  ...
  [ 23  18  26]
  [ 21  19  26]
  [ 20  17  25]]

 ...

 [[143 147 148]
  [143 147 148]
  [150 154 155]
  ...
  [ 40  30  27]
  [ 40  32  22]
  [ 42  34  23]]

 [[137 144 145]
  [139 146 147]
  [143 147 148]
  ...
  [ 38  30  20]
  [ 39  31  21]
  [ 38  30  20]]

 [[138 145 146]
  [139 146 147]
  [131 135 137]
  ...
  [ 37  29  19]
  [ 34  32  17]
  [ 34  32  17]]]
<class 'numpy.ndarray'>
(480, 640, 3)
close close  :  BC
[[[ 51  39  41]
  [ 50  38  40]
  [ 44  36  40]
  ...
  [ 29  25  17]
  [ 26  26  21]
  [ 26  26  21]]

 [[ 57  45  47]
  [ 58  46  48]
  [ 57  43  50]
  ...
  [ 30  26  20]
  [ 28  27  22]
  [ 28  27  22]]

 [[ 57  49  53]
  [ 61  53  56]
  [ 49  40  46]
  ...
  [ 26  26  21]
  [ 23  29  24]


close close  :  BC
[[[ 41  42  38]
  [ 38  40  35]
  [ 61  52  65]
  ...
  [ 17  29  15]
  [ 18  27  21]
  [ 17  26  20]]

 [[ 42  40  45]
  [ 41  39  44]
  [ 52  45  60]
  ...
  [ 18  29  18]
  [ 20  27  21]
  [ 20  27  21]]

 [[ 41  44  50]
  [ 39  41  48]
  [ 47  41  58]
  ...
  [ 20  27  26]
  [ 17  24  23]
  [ 17  24  23]]

 ...

 [[146 146 168]
  [148 147 170]
  [141 145 160]
  ...
  [ 50  37  41]
  [ 43  30  34]
  [ 43  30  34]]

 [[143 144 162]
  [142 142 161]
  [139 142 159]
  ...
  [ 43  30  34]
  [ 42  29  33]
  [ 43  30  34]]

 [[139 142 159]
  [134 137 155]
  [130 131 149]
  ...
  [ 42  29  31]
  [ 40  27  32]
  [ 42  29  33]]]
<class 'numpy.ndarray'>
(480, 640, 3)
close close  :  BC
[[[ 41  43  70]
  [ 41  43  70]
  [ 51  42  69]
  ...
  [ 10  26  26]
  [ 18  25  26]
  [ 18  25  26]]

 [[ 41  44  69]
  [ 47  50  75]
  [ 46  43  66]
  ...
  [ 17  24  25]
  [ 18  25  26]
  [ 17  24  25]]

 [[ 49  50  75]
  [ 41  42  66]
  [ 36  35  58]
  ...
  [ 18  22  24]
  [ 19  23  25]


[[[ 48  47  65]
  [ 46  44  63]
  [ 39  38  61]
  ...
  [ 17  17  17]
  [ 13  21  13]
  [ 15  22  14]]

 [[ 47  40  59]
  [ 47  40  59]
  [ 41  40  58]
  ...
  [ 17  19  14]
  [ 17  20  13]
  [ 17  20  13]]

 [[ 51  42  62]
  [ 50  41  60]
  [ 47  40  59]
  ...
  [ 17  22  21]
  [ 16  22  17]
  [ 16  22  17]]

 ...

 [[152 152 166]
  [146 146 160]
  [147 146 162]
  ...
  [ 51  32  35]
  [ 51  32  35]
  [ 49  29  33]]

 [[152 152 166]
  [148 148 162]
  [151 146 161]
  ...
  [ 51  36  38]
  [ 53  36  38]
  [ 50  33  36]]

 [[146 146 160]
  [146 146 160]
  [147 142 156]
  ...
  [ 46  29  31]
  [ 46  31  35]
  [ 46  31  35]]]
<class 'numpy.ndarray'>
(480, 640, 3)
close close  :  BC
[[[ 86  51  73]
  [ 82  48  69]
  [ 73  44  62]
  ...
  [ 27  27  27]
  [ 27  30  20]
  [ 24  28  18]]

 [[ 77  47  68]
  [ 77  47  68]
  [ 71  47  64]
  ...
  [ 20  27  21]
  [ 27  30  20]
  [ 24  28  18]]

 [[ 72  49  64]
  [ 70  46  62]
  [ 72  48  65]
  ...
  [ 20  25  24]
  [ 22  29  21]
  [ 20  28  20]]

 

[[[ 51  42  62]
  [ 40  32  51]
  [ 36  28  52]
  ...
  [ 15  26  13]
  [ 23  27  15]
  [ 20  24  12]]

 [[ 47  40  59]
  [ 37  31  50]
  [ 41  34  54]
  ...
  [ 25  25  18]
  [ 22  26  14]
  [ 22  26  14]]

 [[ 43  34  54]
  [ 43  34  54]
  [ 45  39  58]
  ...
  [ 22  22  22]
  [ 22  25  17]
  [ 21  23  16]]

 ...

 [[142 141 164]
  [141 140 163]
  [148 140 164]
  ...
  [ 31  28  24]
  [ 28  28  21]
  [ 28  28  21]]

 [[143 142 165]
  [143 142 165]
  [140 133 152]
  ...
  [ 28  25  21]
  [ 26  24  17]
  [ 26  24  17]]

 [[141 138 161]
  [132 129 152]
  [126 119 138]
  ...
  [ 28  25  21]
  [ 28  26  20]
  [ 28  26  20]]]
<class 'numpy.ndarray'>
(480, 640, 3)
close close  :  BC
[[[ 53  42  62]
  [ 52  40  60]
  [ 43  38  53]
  ...
  [ 31  22  15]
  [ 32  21  13]
  [ 31  20  12]]

 [[ 49  41  53]
  [ 53  44  57]
  [ 46  36  51]
  ...
  [ 27  24  13]
  [ 30  22  12]
  [ 29  21  11]]

 [[ 50  39  52]
  [ 47  36  49]
  [ 47  35  50]
  ...
  [ 32  24  14]
  [ 29  26  15]
  [ 29  26  15]]

 

[[[ 50  45  39]
  [ 49  44  38]
  [ 54  40  42]
  ...
  [ 16  18  33]
  [ 22  22  22]
  [ 20  20  20]]

 [[ 51  41  36]
  [ 51  41  36]
  [ 53  38  41]
  ...
  [ 20  22  35]
  [ 23  28  27]
  [ 20  25  24]]

 [[ 48  37  30]
  [ 49  38  31]
  [ 47  33  35]
  ...
  [ 29  31  44]
  [ 27  32  31]
  [ 29  33  33]]

 ...

 [[148 152 167]
  [148 152 167]
  [147 150 168]
  ...
  [ 37  30  31]
  [ 35  30  26]
  [ 36  31  27]]

 [[149 152 170]
  [148 151 169]
  [149 153 168]
  ...
  [ 35  28  29]
  [ 32  26  22]
  [ 36  31  27]]

 [[147 151 166]
  [146 150 165]
  [147 150 168]
  ...
  [ 34  27  28]
  [ 30  23  24]
  [ 34  27  28]]]
<class 'numpy.ndarray'>
(480, 640, 3)
[[[ 62  33  44]
  [ 64  35  47]
  [ 51  30  50]
  ...
  [ 22  21  24]
  [ 19  24  23]
  [ 17  22  21]]

 [[ 60  31  49]
  [ 61  32  50]
  [ 52  32  49]
  ...
  [ 24  24  26]
  [ 19  24  23]
  [ 20  25  24]]

 [[ 55  31  46]
  [ 55  31  46]
  [ 53  36  46]
  ...
  [ 24  24  26]
  [ 20  25  24]
  [ 23  28  27]]

 ...

 [[120 112 136

[[[ 60  36  58]
  [ 62  38  60]
  [ 41  34  54]
  ...
  [ 23  21  28]
  [ 24  23  28]
  [ 23  22  26]]

 [[ 39  26  44]
  [ 41  28  46]
  [ 37  26  52]
  ...
  [ 23  22  25]
  [ 18  21  25]
  [ 19  22  26]]

 [[ 44  32  52]
  [ 42  31  51]
  [ 39  30  50]
  ...
  [ 23  23  23]
  [ 23  23  23]
  [ 22  22  22]]

 ...

 [[138 150 162]
  [141 153 164]
  [145 152 164]
  ...
  [ 37  32  33]
  [ 48  28  31]
  [ 49  29  33]]

 [[141 148 161]
  [142 150 162]
  [144 152 162]
  ...
  [ 39  31  33]
  [ 43  31  33]
  [ 43  31  33]]

 [[142 150 160]
  [141 149 159]
  [142 150 160]
  ...
  [ 34  31  27]
  [ 40  30  27]
  [ 42  31  28]]]
<class 'numpy.ndarray'>
(480, 640, 3)
[[[ 48  41  42]
  [ 47  40  41]
  [ 41  38  46]
  ...
  [ 20  24  26]
  [ 22  22  22]
  [ 23  23  23]]

 [[ 43  34  42]
  [ 46  36  44]
  [ 45  43  50]
  ...
  [ 17  24  23]
  [ 17  24  23]
  [ 18  26  24]]

 [[ 46  36  51]
  [ 47  37  53]
  [ 51  44  58]
  ...
  [ 16  26  24]
  [ 16  26  24]
  [ 16  26  24]]

 ...

 [[149 154 160

open open  :  OO
[[[ 36  45  46]
  [ 37  46  47]
  [ 41  39  44]
  ...
  [ 25  22  18]
  [ 20  21  17]
  [ 17  19  14]]

 [[ 37  46  47]
  [ 37  46  47]
  [ 41  44  50]
  ...
  [ 24  21  17]
  [ 22  21  17]
  [ 22  21  17]]

 [[ 34  44  49]
  [ 34  44  49]
  [ 39  41  48]
  ...
  [ 28  21  22]
  [ 25  23  23]
  [ 25  23  23]]

 ...

 [[146 153 166]
  [146 153 166]
  [140 152 163]
  ...
  [ 41  27  29]
  [ 39  27  29]
  [ 38  26  28]]

 [[148 160 171]
  [144 156 168]
  [143 155 167]
  ...
  [ 46  30  28]
  [ 44  28  30]
  [ 44  28  30]]

 [[141 153 164]
  [142 154 166]
  [142 154 166]
  ...
  [ 42  30  27]
  [ 44  28  30]
  [ 44  28  30]]]
<class 'numpy.ndarray'>
(480, 640, 3)
close close  :  BC
[[[ 50  39  52]
  [ 47  36  49]
  [ 39  27  42]
  ...
  [ 18  26  27]
  [ 31  31  31]
  [ 17  17  17]]

 [[ 46  37  50]
  [ 38  29  42]
  [ 37  26  39]
  ...
  [ 18  26  24]
  [ 25  30  29]
  [ 19  24  23]]

 [[ 39  32  47]
  [ 36  28  43]
  [ 45  36  49]
  ...
  [ 22  22  22]
  [ 24  24  24]
  

In [10]:
     
'''LOGGING IN WITH PASSKEY !!!!! '''


start = input("\n\nDo You Wish to login ? Enter 's' start the processing password and 'q' to stop recording password. Or enter Space to exit.\n\n")

count = 10
k=0

if start=='s':
    cap=cv2.VideoCapture(0)
    login_password = []
    
    index = 0
    global last_code
    
    last_code = ''
    counter = 0
    
    while True:
        time.sleep(0.1)
        ret,frame = cap.read()
        #frame = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        
        frame_temp = frame.copy()
        
        if counter == 0:
            f_x = frame.shape[0]
            f_y = frame.shape[1]
        if cv2.waitKey(1) == ord('q'):
            break
    
        detect_left_and_right_eyes(frame)
        detect_open_eyes(frame)
        
        cv2.imshow('EYE BLINKING AUTHENTICATION',frame_temp)
        
        #Look for the closed eyes
        if l_found == True and r_found == True:
        
            if(left_eye_state == "open" and right_eye_state=="open"):
                Tempstate = 'OO' #
            if(left_eye_state == "open" and right_eye_state=="close"):
                Tempstate = 'RC'
            if(left_eye_state == "close" and right_eye_state=="open"):
                Tempstate = 'LC'
            if(left_eye_state == "close" and right_eye_state=="close"):
                Tempstate = 'BC'
            
            temp_passkey = Tempstate
            print(left_eye_state,right_eye_state," : ",temp_passkey)
            
            
            if index == 0:
                if k<count:
                    k+=1
                    continue
                k=0
                
                login_password.append(temp_passkey)
                last_code = temp_passkey
                index+=1
             
            elif index!=0 and temp_passkey!=last_code:
                if k<count:
                    k+=1
                    continue
                k=0
                login_password.append(temp_passkey)
                last_code = temp_passkey
                index+=1
            
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
      
    cap.release()
    cv2.destroyAllWindows()
    
    print("\n\nLOGIN PASSWORD:   ",login_password)
    
    match = False
    if len(password)==len(login_password):
        for i in range(len(password)):
            if password[i] == login_password[i]:
                match = True
            else:
                match = False
                break
    else:
        match = False
        
    if match == True:
        print("\n\nLOGGED IN YAYAYAY !!!!!!!!\n\n")
    else:
        print("\n\nWRONG PASSWORD!!!\nLOG IN FAILED !!!!!!!!\n\n")



Do You Wish to login ? Enter 's' start the processing password and 'q' to stop recording password. Or enter Space to exit.

s
open close  :  RC
open open  :  OO
open close  :  RC
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open open  :  OO
open close  :  RC
close close  :  BC
close close  :  BC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  RC
open close  :  R